In [3]:
#from benchmark_solvers import greedy_tsp,exact_solver
#from model import MambaFull
import torch
import torch.nn
#import matplotlib.pyplot as plt

coord_dim = 2
city_count = 5
test_size=2000
device = 'cpu'
test_data_loc=f'mamba/data/start_2/test_rand_{test_size}_{city_count}_{coord_dim}.pt'
#test_data = torch.load(test_data_loc).to(device)

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

checkpoint = torch.load('mamba/checkpoints/embed/Linear_mlp_4lay_20-06_13-11.pt')
checkpoint2 = torch.load('mamba/checkpoints/embed/Linear2_18-06_12-57.pt')
checkpoint3 = torch.load('mamba/checkpoints/embed/Linear3_18-06_13-49.pt')

args = checkpoint['args']
print(args)
'''args.mlp_cls = 'identity'
model_train = MambaFull(args.d_model, args.city_count, args.nb_layers, args.coord_dim, args.mlp_cls).to(device)
model_train.load_state_dict(checkpoint['model_state_dict'])
model_train.eval()


mean_tour_length_list = [tensor.cpu().numpy() for tensor in checkpoint['mean_tour_length_list']]
mean_tour_length_list2 = [tensor.cpu().numpy() for tensor in checkpoint2['mean_tour_length_list']]
mean_tour_length_list3 = [tensor.cpu().numpy() for tensor in checkpoint3['mean_tour_length_list']]
print(checkpoint['epoch'])
print(checkpoint['time_tot'])
plt.plot(mean_tour_length_list)
plt.plot(mean_tour_length_list2)
plt.plot(mean_tour_length_list3)

greedy = greedy_tsp(test_data)[0].item()
exact = exact_solver(test_data).item()


plt.axhline(y=greedy, color='r', linestyle='--', label='Greedy Solver')
plt.axhline(y=exact, color='g', linestyle='--', label='Exact Solver')

# Add labels to the axes
plt.xlim(0, 300)
plt.xlabel('Epoch')
plt.ylabel('Mean Tour Length')

plt.legend()
plt.savefig('mamba/figs/mean_tour_length_RL.pdf')
plt.show()

#print(greedy_tsp(test_data_loc))
#print(exact_solver(test_data_loc))'''

{'bsz': 600, 'd_model': 64, 'coord_dim': 2, 'nb_layers': 4, 'mlp_cls': 'gatedmlp', 'city_count': 5, 'deterministic': False, 'fourier_scale': None, 'nb_epochs': 500, 'test_size': 2000, 'nb_batch_per_epoch': 10, 'save_loc': 'checkpoints/embed/Linear_mlp_4lay', 'test_data_loc': 'data/start_2/test_rand_2000_5_2.pt', 'recycle_data': 0, 'checkpoint': None, 'B': None}


"args.mlp_cls = 'identity'\nmodel_train = MambaFull(args.d_model, args.city_count, args.nb_layers, args.coord_dim, args.mlp_cls).to(device)\nmodel_train.load_state_dict(checkpoint['model_state_dict'])\nmodel_train.eval()\n\n\nmean_tour_length_list = [tensor.cpu().numpy() for tensor in checkpoint['mean_tour_length_list']]\nmean_tour_length_list2 = [tensor.cpu().numpy() for tensor in checkpoint2['mean_tour_length_list']]\nmean_tour_length_list3 = [tensor.cpu().numpy() for tensor in checkpoint3['mean_tour_length_list']]\nprint(checkpoint['epoch'])\nprint(checkpoint['time_tot'])\nplt.plot(mean_tour_length_list)\nplt.plot(mean_tour_length_list2)\nplt.plot(mean_tour_length_list3)\n\ngreedy = greedy_tsp(test_data)[0].item()\nexact = exact_solver(test_data).item()\n\n\nplt.axhline(y=greedy, color='r', linestyle='--', label='Greedy Solver')\nplt.axhline(y=exact, color='g', linestyle='--', label='Exact Solver')\n\n# Add labels to the axes\nplt.xlim(0, 300)\nplt.xlabel('Epoch')\nplt.ylabel('Mean 